# Run PPO on Ant from [here](https://github.com/pat-coady/trpo)
#### More descriptions can be found on Patrick Coady's blog regarding [gym and ppo](https://learningai.io/projects/2017/07/28/ai-gym-workout.html) or [descriptions about Ant env](https://gist.github.com/pat-coady/bac60888f011199aad72d2f1e6f5a4fa)

In [1]:
import gym,mujoco_py,warnings,time,os,glob,shutil,csv,skvideo.io
gym.logger.set_level(40)
warnings.filterwarnings("ignore") 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from gym.envs import mujoco
from datetime import datetime
from util import PID_class,Scaler,Logger,display_frames_as_gif
from custom_ant import AntEnvCustom
from ppo import NNValueFunction,Policy,run_episode,run_policy,add_value,discount,\
    add_disc_sum_rew,add_gae,build_train_set,log_batch_stats,run_episode_vid
np.set_printoptions(precision=2,linewidth=150)
%matplotlib inline  
%config InlineBackend.figure_format = 'retina'
print ("Packages Loaded") 

Packages Loaded


### main

In [2]:
env = AntEnvCustom()
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]
env.reset() # Reset 
# render_img = env.render(mode='rgb_array')
print ("obs_dim:[%d] act_dim:[%d]"%(obs_dim,act_dim))

obs_dim += 1  # add 1 to obs dimension for time step feature (see run_episode())
# Logger
env_name = 'Ant'
now = datetime.utcnow().strftime("%b-%d_%H:%M:%S")  # create unique directories
logger = Logger(logName=env_name,now=now,_NOTUSE=True)
aigym_path = os.path.join('/tmp', env_name, now)
# Scaler
scaler = Scaler(obs_dim)
# Value function
hid1_mult = 10
val_func = NNValueFunction(obs_dim, hid1_mult)
# Policy Function
kl_targ = 0.003
policy_logvar = -1.0
policy = Policy(obs_dim, act_dim, kl_targ, hid1_mult, policy_logvar)

Custom Ant Environment made by SJ.
obs_dim:[111] act_dim:[8]
Value Params -- h1: 1120, h2: 74, h3: 5, lr: 0.00116
Policy Params -- h1: 1120, h2: 299, h3: 80, lr: 5.2e-05, logvar_speed: 16
setting up loss with KL penalty


### Run policy for the first time

In [3]:
trajectories = run_policy(env, policy, scaler, logger, episodes=5)
add_value(trajectories, val_func)  # add estimated values to episodes
gamma = 0.995 # Discount factor 
lam = 0.95 # Lambda for GAE
add_disc_sum_rew(trajectories, gamma)  # calculated discounted sum of Rs
add_gae(trajectories, gamma, lam)  # calculate advantage
print ('observes shape:',trajectories[0]['observes'].shape)
print ('actions shape:',trajectories[0]['actions'].shape)
print ('rewards shape:',trajectories[0]['rewards'].shape)
print ('unscaled_obs shape:',trajectories[0]['unscaled_obs'].shape)
print ('values shape:',trajectories[0]['values'].shape)
print ('disc_sum_rew shape:',trajectories[0]['disc_sum_rew'].shape)
print ('advantages shape:',trajectories[0]['advantages'].shape)

observes shape: (42, 112)
actions shape: (42, 8)
rewards shape: (42,)
unscaled_obs shape: (42, 112)
values shape: (42,)
disc_sum_rew shape: (42,)
advantages shape: (42,)


### Loop

In [4]:
SAVE_VID = True
MAKE_GIF = False 

In [ ]:
maxEpoch  = 10000
batchSize = 50
for _epoch in range(maxEpoch):
    # 1. Run policy
    trajectories = run_policy(env, policy, scaler, logger, episodes=batchSize)
    # 2. Get (predict) value from the critic network 
    add_value(trajectories, val_func)  # add estimated values to episodes
    # 3. Get GAE
    gamma = 0.995 # Discount factor 
    lam = 0.95 # Lambda for GAE
    add_disc_sum_rew(trajectories, gamma)  # calculated discounted sum of Rs
    add_gae(trajectories, gamma, lam)  # calculate advantage
    # concatenate all episodes into single NumPy arrays
    observes, actions, advantages, disc_sum_rew = build_train_set(trajectories)
    # add various stats to training log:
    # log_batch_stats(observes, actions, advantages, disc_sum_rew, logger, episode)
    # Update
    policy.update(observes, actions, advantages, logger)  # update policy
    val_func.fit(observes, disc_sum_rew, logger)  # update value function
    # logger.write(display=True)  # write logger results to file and stdout
    
    # Print
    for _tIdx in range(len(trajectories)):
        rs = trajectories[_tIdx]['rewards']
        if _tIdx == 0: rTotal = rs
        else: rTotal = np.concatenate((rTotal,rs))
        # Reward details      
    reward_contacts,reward_ctrls,reward_forwards,reward_headings,reward_survives = [],[],[],[],[]
    tickSum = 0
    for traj in trajectories:
        tickSum += traj['rewards'].shape[0]
        cTraj = traj['rDetails']
        for _iIdx in range(len(cTraj)):
            reward_contacts.append(cTraj[_iIdx]['reward_contact'])
            reward_ctrls.append(cTraj[_iIdx]['reward_ctrl'])
            reward_forwards.append(cTraj[_iIdx]['reward_forward'])
            reward_headings.append(cTraj[_iIdx]['reward_heading'])
            reward_survives.append(cTraj[_iIdx]['reward_survive'])
    tickAvg = tickSum / batchSize
    sumRwd = rTotal.sum() / batchSize
    sumReward_contact = np.asarray(reward_contacts).sum() / batchSize
    sumReward_ctrl = np.asarray(reward_ctrls).sum() / batchSize
    sumReward_forward = np.asarray(reward_forwards).sum() / batchSize
    sumReward_heading = np.asarray(reward_headings).sum() / batchSize
    sumReward_survive = np.asarray(reward_survives).sum() / batchSize
    print ("[%d/%d](#total:%d) sumRwd:[%.3f](cntct:%.3f+ctrl:%.3f+fwd:%.3f+head:%.3f+srv:%.3f) tickAvg:[%d]"%
           (_epoch,maxEpoch,(_epoch+1)*batchSize,sumRwd,
           sumReward_contact,sumReward_ctrl,sumReward_forward,sumReward_heading,sumReward_survive,tickAvg))
    
    # SHOW EVERY 
    PLOT_EVERY = 20 
    DO_ANIMATE = False
    if ((_epoch%PLOT_EVERY)==0 ) | (_epoch==(maxEpoch-1)):
        ret = run_episode_vid(env, policy, scaler)
        print ("  [^] sumRwd:[%.3f] Xdisp:[%.3f] hDisp:[%.1f]"%
               (np.asarray(ret['rewards']).sum(),ret['xDisp'],ret['hDisp']))
        if MAKE_GIF:
            display_frames_as_gif(ret['frames'])
        if SAVE_VID:
            outputdata = np.asarray(ret['frames']).astype(np.uint8)
            vidName = 'vids/ant_ppo_epoch%03d.mp4'%(_epoch)
            skvideo.io.vwrite(vidName,outputdata)
            print ("[%s] saved."%(vidName))
print ("Done.") 

[0/10000](#total:50) sumRwd:[-129.968](cntct:-0.083+ctrl:-149.131+fwd:-9.885+head:-47.769+srv:76.900) tickAvg:[76]
Creating window glfw
  [^] sumRwd:[-33.288] Xdisp:[-0.314] hDisp:[-67.1]
[vids/ant_ppo_epoch000.mp4] saved.
[1/10000](#total:100) sumRwd:[-138.597](cntct:-0.093+ctrl:-167.291+fwd:-7.121+head:-51.072+srv:86.980) tickAvg:[86]
[2/10000](#total:150) sumRwd:[-151.172](cntct:-0.090+ctrl:-165.621+fwd:-7.401+head:-63.540+srv:85.480) tickAvg:[85]
[3/10000](#total:200) sumRwd:[-130.932](cntct:-0.085+ctrl:-151.000+fwd:-11.546+head:-46.801+srv:78.500) tickAvg:[78]
[4/10000](#total:250) sumRwd:[-176.840](cntct:-0.100+ctrl:-186.039+fwd:-12.293+head:-72.548+srv:94.140) tickAvg:[94]
[5/10000](#total:300) sumRwd:[-117.027](cntct:-0.081+ctrl:-149.985+fwd:-1.702+head:-42.759+srv:77.500) tickAvg:[77]
[6/10000](#total:350) sumRwd:[-131.626](cntct:-0.081+ctrl:-151.955+fwd:-5.846+head:-51.524+srv:77.780) tickAvg:[77]
[7/10000](#total:400) sumRwd:[-150.776](cntct:-0.084+ctrl:-164.582+fwd:-8.018+h

[67/10000](#total:3400) sumRwd:[-78.542](cntct:-0.046+ctrl:-99.030+fwd:-6.831+head:-21.655+srv:49.020) tickAvg:[49]
[68/10000](#total:3450) sumRwd:[-71.222](cntct:-0.043+ctrl:-90.022+fwd:-4.598+head:-20.539+srv:43.980) tickAvg:[43]
[69/10000](#total:3500) sumRwd:[-77.026](cntct:-0.051+ctrl:-102.843+fwd:-5.077+head:-19.455+srv:50.400) tickAvg:[50]
[70/10000](#total:3550) sumRwd:[-85.389](cntct:-0.050+ctrl:-104.115+fwd:-5.388+head:-25.876+srv:50.040) tickAvg:[50]
[71/10000](#total:3600) sumRwd:[-75.450](cntct:-0.048+ctrl:-96.045+fwd:-9.946+head:-16.672+srv:47.260) tickAvg:[47]
[72/10000](#total:3650) sumRwd:[-78.464](cntct:-0.048+ctrl:-98.079+fwd:-9.568+head:-19.190+srv:48.420) tickAvg:[48]
[73/10000](#total:3700) sumRwd:[-85.360](cntct:-0.051+ctrl:-104.686+fwd:-2.376+head:-30.108+srv:51.860) tickAvg:[51]
[74/10000](#total:3750) sumRwd:[-97.919](cntct:-0.056+ctrl:-110.486+fwd:-9.993+head:-30.984+srv:53.600) tickAvg:[53]
[75/10000](#total:3800) sumRwd:[-79.982](cntct:-0.045+ctrl:-95.148+f

[135/10000](#total:6800) sumRwd:[-70.870](cntct:-0.040+ctrl:-89.950+fwd:-4.833+head:-18.107+srv:42.060) tickAvg:[42]
[136/10000](#total:6850) sumRwd:[-105.258](cntct:-0.047+ctrl:-103.230+fwd:-1.557+head:-47.444+srv:47.020) tickAvg:[47]
[137/10000](#total:6900) sumRwd:[-69.591](cntct:-0.041+ctrl:-94.482+fwd:-2.755+head:-16.413+srv:44.100) tickAvg:[44]
[138/10000](#total:6950) sumRwd:[-87.376](cntct:-0.047+ctrl:-98.558+fwd:-6.078+head:-28.113+srv:45.420) tickAvg:[45]
[139/10000](#total:7000) sumRwd:[-70.170](cntct:-0.036+ctrl:-86.538+fwd:-4.085+head:-21.011+srv:41.500) tickAvg:[41]
[140/10000](#total:7050) sumRwd:[-69.004](cntct:-0.041+ctrl:-90.590+fwd:-1.238+head:-19.014+srv:41.880) tickAvg:[41]
  [^] sumRwd:[-10.935] Xdisp:[0.548] hDisp:[43.0]
[vids/ant_ppo_epoch140.mp4] saved.
[141/10000](#total:7100) sumRwd:[-81.165](cntct:-0.049+ctrl:-104.842+fwd:-4.491+head:-18.143+srv:46.360) tickAvg:[46]
[142/10000](#total:7150) sumRwd:[-75.964](cntct:-0.045+ctrl:-98.015+fwd:-2.223+head:-19.661+s

[203/10000](#total:10200) sumRwd:[-39.566](cntct:-0.021+ctrl:-56.754+fwd:-2.260+head:-5.711+srv:25.180) tickAvg:[25]
[204/10000](#total:10250) sumRwd:[-48.550](cntct:-0.030+ctrl:-76.076+fwd:2.074+head:-9.177+srv:34.660) tickAvg:[34]
[205/10000](#total:10300) sumRwd:[-66.083](cntct:-0.031+ctrl:-80.586+fwd:-3.302+head:-17.224+srv:35.060) tickAvg:[35]
[206/10000](#total:10350) sumRwd:[-54.616](cntct:-0.026+ctrl:-70.550+fwd:-3.154+head:-12.785+srv:31.900) tickAvg:[31]
[207/10000](#total:10400) sumRwd:[-60.186](cntct:-0.034+ctrl:-83.437+fwd:-1.630+head:-10.925+srv:35.840) tickAvg:[35]
[208/10000](#total:10450) sumRwd:[-63.435](cntct:-0.038+ctrl:-88.768+fwd:-0.083+head:-14.606+srv:40.060) tickAvg:[40]
[209/10000](#total:10500) sumRwd:[-55.686](cntct:-0.029+ctrl:-73.339+fwd:-1.134+head:-14.983+srv:33.800) tickAvg:[33]
[210/10000](#total:10550) sumRwd:[-66.251](cntct:-0.034+ctrl:-83.450+fwd:-5.960+head:-14.549+srv:37.740) tickAvg:[37]
[211/10000](#total:10600) sumRwd:[-93.397](cntct:-0.048+ctr

[271/10000](#total:13600) sumRwd:[-59.372](cntct:-0.033+ctrl:-80.853+fwd:-1.086+head:-12.060+srv:34.660) tickAvg:[34]
[272/10000](#total:13650) sumRwd:[-62.927](cntct:-0.034+ctrl:-86.354+fwd:1.435+head:-14.674+srv:36.700) tickAvg:[36]
[273/10000](#total:13700) sumRwd:[-105.971](cntct:-0.049+ctrl:-118.004+fwd:-0.889+head:-36.789+srv:49.760) tickAvg:[49]
[274/10000](#total:13750) sumRwd:[-45.145](cntct:-0.024+ctrl:-63.107+fwd:-1.307+head:-8.267+srv:27.560) tickAvg:[27]
[275/10000](#total:13800) sumRwd:[-49.155](cntct:-0.027+ctrl:-69.264+fwd:2.024+head:-11.608+srv:29.720) tickAvg:[29]
[276/10000](#total:13850) sumRwd:[-46.589](cntct:-0.027+ctrl:-67.076+fwd:1.551+head:-9.977+srv:28.940) tickAvg:[28]
[277/10000](#total:13900) sumRwd:[-67.592](cntct:-0.037+ctrl:-86.191+fwd:-1.727+head:-15.178+srv:35.540) tickAvg:[35]
[278/10000](#total:13950) sumRwd:[-52.771](cntct:-0.029+ctrl:-77.323+fwd:1.121+head:-10.180+srv:33.640) tickAvg:[33]
[279/10000](#total:14000) sumRwd:[-70.557](cntct:-0.038+ctrl

...

[4817/10000](#total:240900) sumRwd:[336.056](cntct:-0.243+ctrl:-23.500+fwd:179.368+head:-0.569+srv:181.000) tickAvg:[181]
[4818/10000](#total:240950) sumRwd:[336.033](cntct:-0.245+ctrl:-23.253+fwd:179.057+head:-0.526+srv:181.000) tickAvg:[181]
[4819/10000](#total:241000) sumRwd:[336.255](cntct:-0.244+ctrl:-23.292+fwd:179.311+head:-0.520+srv:181.000) tickAvg:[181]
[4820/10000](#total:241050) sumRwd:[336.252](cntct:-0.240+ctrl:-23.142+fwd:179.188+head:-0.554+srv:181.000) tickAvg:[181]
  [^] sumRwd:[334.702] Xdisp:[13.639] hDisp:[-4.3]
[vids/ant_ppo_epoch4820.mp4] saved.
[4821/10000](#total:241100) sumRwd:[336.478](cntct:-0.243+ctrl:-23.164+fwd:179.362+head:-0.477+srv:181.000) tickAvg:[181]
[4822/10000](#total:241150) sumRwd:[335.882](cntct:-0.241+ctrl:-23.488+fwd:179.177+head:-0.566+srv:181.000) tickAvg:[181]
[4823/10000](#total:241200) sumRwd:[324.355](cntct:-0.242+ctrl:-29.183+fwd:175.978+head:-3.198+srv:181.000) tickAvg:[181]
[4824/10000](#total:241250) sumRwd:[336.308](cntct:-0.237+c

[4882/10000](#total:244150) sumRwd:[336.183](cntct:-0.238+ctrl:-23.049+fwd:179.091+head:-0.621+srv:181.000) tickAvg:[181]
[4883/10000](#total:244200) sumRwd:[336.249](cntct:-0.245+ctrl:-22.792+fwd:178.861+head:-0.574+srv:181.000) tickAvg:[181]
[4884/10000](#total:244250) sumRwd:[335.809](cntct:-0.241+ctrl:-23.138+fwd:178.832+head:-0.644+srv:181.000) tickAvg:[181]
[4885/10000](#total:244300) sumRwd:[336.215](cntct:-0.241+ctrl:-23.053+fwd:179.152+head:-0.643+srv:181.000) tickAvg:[181]
[4886/10000](#total:244350) sumRwd:[336.424](cntct:-0.238+ctrl:-23.121+fwd:179.445+head:-0.662+srv:181.000) tickAvg:[181]
[4887/10000](#total:244400) sumRwd:[336.146](cntct:-0.239+ctrl:-23.226+fwd:179.262+head:-0.652+srv:181.000) tickAvg:[181]
[4888/10000](#total:244450) sumRwd:[336.142](cntct:-0.242+ctrl:-23.070+fwd:179.042+head:-0.588+srv:181.000) tickAvg:[181]
[4889/10000](#total:244500) sumRwd:[336.656](cntct:-0.240+ctrl:-22.970+fwd:179.444+head:-0.578+srv:181.000) tickAvg:[181]
[4890/10000](#total:2445

[4948/10000](#total:247450) sumRwd:[337.635](cntct:-0.240+ctrl:-21.716+fwd:179.171+head:-0.580+srv:181.000) tickAvg:[181]
[4949/10000](#total:247500) sumRwd:[337.287](cntct:-0.245+ctrl:-21.978+fwd:179.066+head:-0.556+srv:181.000) tickAvg:[181]
[4950/10000](#total:247550) sumRwd:[337.109](cntct:-0.240+ctrl:-22.082+fwd:178.923+head:-0.492+srv:181.000) tickAvg:[181]
[4951/10000](#total:247600) sumRwd:[337.722](cntct:-0.240+ctrl:-21.656+fwd:179.150+head:-0.532+srv:181.000) tickAvg:[181]
[4952/10000](#total:247650) sumRwd:[337.457](cntct:-0.243+ctrl:-21.719+fwd:178.935+head:-0.517+srv:181.000) tickAvg:[181]
[4953/10000](#total:247700) sumRwd:[337.493](cntct:-0.243+ctrl:-21.809+fwd:179.051+head:-0.505+srv:181.000) tickAvg:[181]
[4954/10000](#total:247750) sumRwd:[337.903](cntct:-0.241+ctrl:-21.718+fwd:179.337+head:-0.475+srv:181.000) tickAvg:[181]
[4955/10000](#total:247800) sumRwd:[337.440](cntct:-0.243+ctrl:-21.864+fwd:179.128+head:-0.582+srv:181.000) tickAvg:[181]
[4956/10000](#total:2478

[5014/10000](#total:250750) sumRwd:[338.244](cntct:-0.244+ctrl:-20.998+fwd:179.017+head:-0.531+srv:181.000) tickAvg:[181]
[5015/10000](#total:250800) sumRwd:[338.581](cntct:-0.237+ctrl:-20.845+fwd:179.201+head:-0.537+srv:181.000) tickAvg:[181]
[5016/10000](#total:250850) sumRwd:[338.525](cntct:-0.247+ctrl:-20.913+fwd:179.223+head:-0.537+srv:181.000) tickAvg:[181]
[5017/10000](#total:250900) sumRwd:[337.119](cntct:-0.246+ctrl:-21.279+fwd:178.178+head:-0.534+srv:181.000) tickAvg:[181]
[5018/10000](#total:250950) sumRwd:[338.716](cntct:-0.237+ctrl:-20.716+fwd:179.204+head:-0.535+srv:181.000) tickAvg:[181]
[5019/10000](#total:251000) sumRwd:[338.329](cntct:-0.239+ctrl:-20.941+fwd:179.008+head:-0.500+srv:181.000) tickAvg:[181]
[5020/10000](#total:251050) sumRwd:[338.419](cntct:-0.239+ctrl:-20.992+fwd:179.176+head:-0.527+srv:181.000) tickAvg:[181]
  [^] sumRwd:[339.419] Xdisp:[14.122] hDisp:[-8.5]
[vids/ant_ppo_epoch5020.mp4] saved.
[5021/10000](#total:251100) sumRwd:[337.895](cntct:-0.244+c

[5080/10000](#total:254050) sumRwd:[339.455](cntct:-0.234+ctrl:-20.134+fwd:179.282+head:-0.460+srv:181.000) tickAvg:[181]
  [^] sumRwd:[340.339] Xdisp:[14.996] hDisp:[-3.1]
[vids/ant_ppo_epoch5080.mp4] saved.
[5081/10000](#total:254100) sumRwd:[339.623](cntct:-0.238+ctrl:-20.118+fwd:179.404+head:-0.426+srv:181.000) tickAvg:[181]
[5082/10000](#total:254150) sumRwd:[335.492](cntct:-0.233+ctrl:-19.710+fwd:177.216+head:-0.421+srv:178.640) tickAvg:[178]
[5083/10000](#total:254200) sumRwd:[339.895](cntct:-0.240+ctrl:-20.000+fwd:179.544+head:-0.409+srv:181.000) tickAvg:[181]
[5084/10000](#total:254250) sumRwd:[339.753](cntct:-0.239+ctrl:-20.159+fwd:179.562+head:-0.411+srv:181.000) tickAvg:[181]
[5085/10000](#total:254300) sumRwd:[339.778](cntct:-0.231+ctrl:-20.059+fwd:179.540+head:-0.473+srv:181.000) tickAvg:[181]
[5086/10000](#total:254350) sumRwd:[339.313](cntct:-0.237+ctrl:-20.159+fwd:179.130+head:-0.422+srv:181.000) tickAvg:[181]
[5087/10000](#total:254400) sumRwd:[339.571](cntct:-0.241+c

### Animate final motion

In [ ]:
SAVE_VID_FINAL = True
MAKE_GIF_FINAL = False 

In [ ]:
for _i in range(3):
    ret = run_episode_vid(env, policy, scaler)
    if MAKE_GIF_FINAL:
        display_frames_as_gif(ret['frames'])
    if SAVE_VID_FINAL:
        outputdata = np.asarray(ret['frames']).astype(np.uint8)
        vidName = 'vids/ant_ppo_final_%d.mp4'%(_i)
        skvideo.io.vwrite(vidName,outputdata)
        print ("[%s] saved."%(vidName))

### Finished

In [ ]:
DO_CLOSE = False # There is no turning back. 
if DO_CLOSE:
    logger.close()
    policy.close_sess()
    val_func.close_sess()